In [1]:
import pandas as pd
import wandb

In [2]:
with open("wandb_apikey.txt") as f:
    api_key = f.read().strip()

In [3]:
assert wandb.login(key=api_key)
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: stevenabreu7. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sabreu/.netrc


In [4]:
project = "NeuroSSMs"
entity = "il-ncl"

In [5]:
runs = api.runs(f"{entity}/{project}")

In [6]:
# sMNIST runs
configs = "full W8A8 W4A8Wssm8 W4A8Wa8 W4A8 W2A8Wssm8 W2A8Wa8 W2A8".split(" ")
suffices = ["-nqact", "", "-lnrm"]
data = []
for config in configs:
    d = {}
    d["config"] = f'{config.replace("Wssm8", " (SSM: W8)").replace("Wa8", " ($A$: W8)").replace("full", "Baseline"):15}'
    for suffix in suffices:
        rename = {"-nqact": "GeLU \\& BN", "": "qGeLU \\& BN", "-lnrm": "qGeLU \\& LN"}
        runs = api.runs(f"{entity}/{project}", filters={"displayName": f"smnist-{config}{suffix}"})
        if len(runs) > 1:
            print(f"SMNIST run with {config}{suffix} has multiple matches!!.")
        if len(runs) == 0:
            print(f"SMNIST run with {config}{suffix} was not found.")
            d[rename[suffix]] = f'{"n/a":19}'
        else:
            run = runs[0]
            acc = run.summary["Best Test Accuracy"]
            if acc > 0.99:
                d[rename[suffix]] = '\\textbf{' + f'{acc:.2%}' + '}   '
            elif acc > 0.98:
                d[rename[suffix]] = '\\underline{' + f'{acc:.2%}' + '}'
            else:
                d[rename[suffix]] = f'{acc:<18.2%}'
    data.append(d)
df = pd.DataFrame(data)
df.index = list(df["config"])
df = df.drop(columns="config")
print(df.to_latex().replace("%", "\\%"))

SMNIST run with full-nqact was not found.
\begin{tabular}{llll}
\toprule
 & GeLU \& BN & qGeLU \& BN & qGeLU \& LN \\
\midrule
Baseline        & n/a                 & \textbf{99.53\%}    & \textbf{99.48\%}    \\
W8A8            & \textbf{99.48\%}    & \textbf{99.47\%}    & \textbf{99.54\%}    \\
W4A8 (SSM: W8)  & \textbf{99.52\%}    & \textbf{99.52\%}    & \textbf{99.63\%}    \\
W4A8 ($A$: W8)  & 97.32\%             & \underline{98.03\%} & \textbf{99.26\%}    \\
W4A8            & 9.80\%              & 34.51\%             & 12.68\%             \\
W2A8 (SSM: W8)  & \textbf{99.64\%}    & \textbf{99.45\%}    & \textbf{99.56\%}    \\
W2A8 ($A$: W8)  & 71.63\%             & 82.40\%             & 80.76\%             \\
W2A8            & 46.07\%             & 46.77\%             & 54.75\%             \\
\bottomrule
\end{tabular}

